In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization

from isic_helper import get_folds
from isic_helper import compute_pauc, compute_auc

In [2]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_preds_ensemble"

model_names = ["cb", "lgb", "resnet18", "efficientnet_b0"]
versions = ["v1", "v3", "v2",  "v1"]
paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-train" for model_name, version in zip(model_names, versions)]

In [3]:
val_auc_scores = {}
val_pauc_scores = {}
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    oof_preds_model_df = pd.read_csv(f"{path}/oof_preds_{model_name}_{version}.csv")
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, f"oof_{model_name}_{version}"]],
                                          on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    val_auc_scores[f"{model_name}_{version}"] = {}
    val_pauc_scores[f"{model_name}_{version}"] = {}
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}

In [4]:
oof_columns = [col for col in oof_preds_df.columns if col.startswith("oof")]
all_folds = np.unique(oof_preds_df["fold"])

In [5]:
def blend_optimizer(oof_preds_df, oof_columns, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (0.0, 1.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in all_folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(all_folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=42,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights


weights = blend_optimizer(
    oof_preds_df, oof_columns, 
    init_points=40, 
    n_iter=40
)

|   iter    |  target   |    w0     |    w1     |    w2     |    w3     |
-------------------------------------------------------------------------
| 1         | 0.1731    | 0.3745    | 0.9507    | 0.732     | 0.5987    |
| 2         | 0.1618    | 0.156     | 0.156     | 0.05808   | 0.8662    |
| 3         | 0.1708    | 0.6011    | 0.7081    | 0.02058   | 0.9699    |
| 4         | 0.1727    | 0.8324    | 0.2123    | 0.1818    | 0.1834    |
| 5         | 0.1736    | 0.3042    | 0.5248    | 0.4319    | 0.2912    |
| 6         | 0.1729    | 0.6119    | 0.1395    | 0.2921    | 0.3664    |
| 7         | 0.1732    | 0.4561    | 0.7852    | 0.1997    | 0.5142    |
| 8         | 0.172     | 0.5924    | 0.04645   | 0.6075    | 0.1705    |
| 9         | 0.1706    | 0.06505   | 0.9489    | 0.9656    | 0.8084    |
| 10        | 0.1687    | 0.3046    | 0.09767   | 0.6842    | 0.4402    |
| 11        | 0.1666    | 0.122     | 0.4952    | 0.03439   | 0.9093    |
| 12        | 0.1728    | 0.2588    | 

In [6]:
all_folds = np.unique(oof_preds_df["fold"])
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, weight in zip(model_names, versions, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, f"oof_{model_name}_{version}"]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, weight in zip(model_names, versions, weights):
    print(f"Model: {model_name}_{version} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")

Model: cb_v1 | Weightage: 1.0
Val AUC scores:
{'fold_1': 0.9523881910669536,
 'fold_2': 0.9535923162036922,
 'fold_3': 0.9457296489498446,
 'fold_4': 0.9721251762738515,
 'fold_5': 0.9470801267252167}
Val PAUC scores:
{'fold_1': 0.1645271757829003,
 'fold_2': 0.1669476383467947,
 'fold_3': 0.15652509640463735,
 'fold_4': 0.17601833248056303,
 'fold_5': 0.15814944833701147}
CV AUC OOF: 0.9524292434775488
CV PAUC OOF: 0.16241203053404624
CV AUC AVG: 0.9541830918439118
CV PAUC AVG: 0.16443353827038137
CV AUC STD: 0.009459406837513047
CV PAUC STD: 0.006965201549198192


Model: lgb_v3 | Weightage: 1.0
Val AUC scores:
{'fold_1': 0.9576815612354752,
 'fold_2': 0.9508534742767404,
 'fold_3': 0.9506974516728857,
 'fold_4': 0.9630832636557638,
 'fold_5': 0.9616494948942257}
Val PAUC scores:
{'fold_1': 0.16882108869081178,
 'fold_2': 0.16124885048313303,
 'fold_3': 0.16251325953750728,
 'fold_4': 0.1684323170807802,
 'fold_5': 0.17159456378130328}
CV AUC OOF: 0.9546938253660073
CV PAUC OOF: 0.164

In [7]:
oof_preds_df

,isic_id,patient_id,fold,target,oof_cb_v1,oof_lgb_v3,oof_resnet18_v2,oof_efficientnet_b0_v1,oof_preds_ensemble
0,ISIC_0015670,IP_1235828,4,0,0.000104,0.000015,0.019822,0.013287,1.397054
1,ISIC_0015845,IP_8170065,1,0,0.456523,0.652166,0.481737,0.216192,3.160530
2,ISIC_0015864,IP_6724798,5,0,0.000034,0.000019,0.001913,0.001323,0.862643
3,ISIC_0015902,IP_4111386,2,0,0.000022,0.000024,0.021381,0.000164,1.548688
4,ISIC_0024200,IP_8313778,1,0,0.000522,0.000160,0.026625,0.025489,2.405850
...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,IP_1140263,3,0,0.003126,0.002748,0.092435,0.011951,2.971599
401055,ISIC_9999951,IP_5678181,3,0,0.000053,0.000030,0.000728,0.000092,0.842569
401056,ISIC_9999960,IP_0076153,2,0,0.001993,0.000537,0.002350,0.000105,2.235032
401057,ISIC_9999964,IP_5231513,5,0,0.000071,0.000028,0.002016,0.002244,1.224368


In [8]:
oof_preds_df.pivot_table(index="target", values=oof_columns+["oof_preds_ensemble"], aggfunc="mean")

,oof_cb_v1,oof_efficientnet_b0_v1,oof_lgb_v3,oof_preds_ensemble,oof_resnet18_v2
target,,,,,
0,0.000671,0.068609,0.000313,1.626378,0.086808
1,0.034607,0.577327,0.021116,3.076971,0.611987


In [9]:
oof_preds_df[oof_columns+["oof_preds_ensemble"]].corr()

,oof_cb_v1,oof_lgb_v3,oof_resnet18_v2,oof_efficientnet_b0_v1,oof_preds_ensemble
oof_cb_v1,1.000000,0.757476,0.230135,0.236565,0.199167
oof_lgb_v3,0.757476,1.000000,0.170230,0.169802,0.130706
oof_resnet18_v2,0.230135,0.170230,1.000000,0.663187,0.593230
oof_efficientnet_b0_v1,0.236565,0.169802,0.663187,1.000000,0.520657
oof_preds_ensemble,0.199167,0.130706,0.593230,0.520657,1.000000
